In [ ]:
%matplotlib widget

import ipywidgets as widgets
from ipywidgets import HBox, VBox, jslink, Box, Layout, Output, Label
from IPython.display import display, Latex, Image, Markdown

import params as st
from model import *
from feedforward import *

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from scipy.integrate import solve_ivp

In [ ]:
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 5px 5px 0px',
        padding='2px 2px 2px 2px'
     )

# Steuerung des Magnetlagers

In [ ]:
imag = Image("../images/magnetlager.png", width=125)

outL = Output()
outR = Output()

with outL:
    display(imag)
with outR:
    display(Markdown("""
**Modellgleichungen**

\\begin{align*}
    m \\ddot{y}(t) & = k \\frac{i^2(t)}{(y(t)-s_0)^2} - m g
\\end{align*}

**flachheitsbasierte Steuerung**

Entwurf |   | Eingang
----------------------|----|------------------------
\\begin{align*}
    y_{\\mathrm{r}}(t) & = \\begin{cases}
    y_0, & t \\le t_0 \\\\
    y_0 + y_{\\mathrm{d}} \\varphi_2(t/T) & t_0 < t < t_0 + T \\\\
    y_{\\mathrm{d}} & t \\ge t_0 + T
    \\end{cases}
\\end{align*} |   | \\begin{align*}
    i_{\\mathrm{r}}(t) & = (y_{\\mathrm{r}}(t) - s_0)\\sqrt{\\frac{m}{k} (\\ddot{y}_{\\mathrm{r}}(t) + g)}
\\end{align*}

    """))
cols = HBox([outL, outR], layout=Layout(display='flex', flex_flow='row', justify_content='space-around', align_items='center'))
display(cols)

**Definition Parameter**

In [ ]:
tSim = np.linspace(0, 5, 501)

In [ ]:
output = widgets.Output()

with output:
    fig = plt.figure(figsize=(10, 5))
    ax3 = plt.subplot(222)
    ax4 = plt.subplot(224)
    ax1 = plt.subplot(221)
    ax2 = plt.subplot(223)

plt.subplots_adjust(wspace=0.2, hspace=0.3)
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.subplots_adjust(bottom=0.1, top=0.93, left=0.125, right=0.9)

ax1.set_xlim([tSim[0], tSim[-1]]) 
ax2.set_xlim([tSim[0], tSim[-1]]) 
ax4.set_xlim([tSim[0], tSim[-1]]) 
ax1.set_ylim([-1, 6]) 
ax2.set_ylim([0, 2]) 
ax4.set_ylim([-5, 5]) 
ax1.grid() 
ax2.grid()
ax4.grid()
ax3.set_xlim(-2, 2)
ax3.set_ylim(-1, 3)
ax3.set_xticks([])
ax3.set_yticks([])
ax1.set_ylabel(r"$u$")
ax2.set_xlabel(r"$t$")
ax2.set_ylabel(r"$y$ in m")
ax4.set_ylabel(r"$\dot{y}$ in m/s")
ax4.set_xlabel(r"$t$")

lineY, = ax2.plot([], [], label=r"Istverlauf")
lineYref, = ax2.plot([], [], '--', label=r"Sollverlauf")
linedY, = ax4.plot([], [])
linedYref, = ax4.plot([], [], '--')
lineU, = ax1.plot([], [])

ballAni = ax3.add_patch(Ellipse((0, 0), st.r, st.b, facecolor='0.5', edgecolor='0.25'))
magAni = ax3.add_patch(plt.Rectangle((-st.r, st.r + st.s), 2 * st.r, st.h, facecolor='0.', edgecolor='0.'))

handlesAx, labelsAx = ax2.get_legend_handles_labels()
fig.legend([handle for i, handle in enumerate(handlesAx)],
           [label for i, label in enumerate(labelsAx)],
           bbox_to_anchor=(0.125, 0.94, 0.7735, .15), loc=3,
           ncol=2, mode="expand", borderaxespad=0., framealpha=0.5)

playB = widgets.Play(value=0,
                     min=0, 
                     max=len(tSim),
                     step=10)
sliderB = widgets.IntSlider(value=0,
                            min=0,
                            max=len(tSim),
                            step=10)
sliderT0 = widgets.FloatSlider(value=0,
                               min=0,
                               max=5,
                               step=1,
                               description=r'$t_0$')
sliderT = widgets.FloatSlider(value=1,
                               min=0.5,
                               max=4,
                               step=.5,
                               description=r'$T$')
sliderYd = widgets.widgets.FloatSlider(value=0.5,
                                       min=0.2,
                                       max=0.7,
                                       step=0.1,
                                       description=r"$y_\text{d}$")
sliderY0 = widgets.widgets.FloatSlider(value=1,
                                       min=0.5,
                                       max=1.1,
                                       step=0.1,
                                       description=r"$y_\text{0}$")
radioU = widgets.RadioButtons(options=['Sprung', 'Steuerung: flach'],
                              disabled=False)

def updateOde(_):
    global res

    t0 = sliderT0.value
    T = sliderT.value
    yd = sliderYd.value
    y0 = sliderY0.value

    def sys(t, x, u, params):
        dx = nonlinSys(t, x, u, params)

        return dx

    params = st.g, st.m, st.s0 , st.k
    if radioU.value == 'Sprung':
        uS = stationarySys(y0, params)
        x0 = [y0, 0]
        uIn = feedForwardStep(t0, T, yd, uS)
    else:
        x0 = [y0, 0]
        _, uIn = feedForwardFlat(t0, T, yd, y0)

    res = solve_ivp(sys,
                    [tSim[0], tSim[-1]],
                    x0,
                    t_eval=tSim,
                    args=(uIn, params))

def updatePlot(change):
    idx = change['new']

    y = res.y.T[idx, 0]
    dy = res.y.T[idx, 1]
    
    t0 = sliderT0.value
    T = sliderT.value
    yd = sliderYd.value
    y0 = sliderY0.value

    params = st.g, st.m, st.s0 , st.k
    if radioU.value == 'Sprung':
        uS = stationarySys(y0, params)
        uIn = feedForwardStep(t0, T, yd, uS)
        lineYref.set_data([], [])
        linedYref.set_data([], [])
        yMin = np.min(res.y.T[:, 0])
        yMax = np.max(res.y.T[:, 0])
        dyMin = np.min(res.y.T[:, 1])
        dyMax = np.max(res.y.T[:, 1])
    else:
        yr, uIn = feedForwardFlat(t0, T, yd, y0)
        lineYref.set_data(tSim[:idx], [yr[0](_t) for _t in tSim[:idx]])
        linedYref.set_data(tSim[:idx], [yr[1](_t) for _t in tSim[:idx]])
        yMin = np.minimum(np.min(res.y.T[:, 0]), np.min([yr[0](_t) for _t in tSim]))
        yMax = np.maximum(np.max(res.y.T[:, 0]), np.max([yr[0](_t) for _t in tSim]))
        dyMin = np.minimum(np.min(res.y.T[:, 1]), np.min([yr[1](_t) for _t in tSim]))
        dyMax = np.maximum(np.max(res.y.T[:, 1]), np.max([yr[1](_t) for _t in tSim]))

    linedY.set_data(tSim[:idx], res.y.T[:idx, 1])
    lineU.set_data(tSim[:idx], [uIn(_t) for _t in tSim[:idx]])
    
    uMin = np.min([uIn(_t) for _t in tSim])
    uMax = np.max([uIn(_t) for _t in tSim])
    ax1.set_ylim(uMin - np.abs(uMax - uMin) * 0.1, uMax + np.abs(uMax - uMin) * 0.1)
    ax2.set_ylim(yMin - np.abs(yMax - yMin) * 0.1, yMax + np.abs(yMax - yMin) * 0.1)
    ax4.set_ylim(dyMin - np.abs(dyMax - dyMin) * 0.1, dyMax + np.abs(dyMax - dyMin) * 0.1)

    ballAni.set_center((0, y - 1))
    
    fig.canvas.draw()    

sliderB.observe(updatePlot, names='value')
sliderT0.observe(updateOde, names='value')
sliderT.observe(updateOde, names='value')
sliderYd.observe(updateOde, names='value')
sliderY0.observe(updateOde, names='value')
radioU.observe(updateOde, names='value')

updateOde(_)

bFeedForward = VBox([Label(value='Steuerung'), sliderY0, sliderYd, sliderT0, sliderT])
bFeedForward.layout = make_box_layout()
bInput = VBox([Label(value='Eingang'), radioU])
bInput.layout = make_box_layout()
controls = HBox([bFeedForward, bInput])

jslink((playB, 'value'), (sliderB, 'value'))
videoControls = VBox([HBox([playB, sliderB]), output])
videoControls.layout = make_box_layout()

HBox([controls, Box([videoControls])], layout=Layout(display='flex', flex_flow='column', justify_content='center', align_items='center'))